In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import csv

In [ ]:
# Try scraping with BeautifulSoup only, not Splinter
# executable_path = {'executable_path': '/Users/Bons/Downloads/chromedriver'}
# browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# List of pages to be looped through
url_list = []

# URL of page to be scraped
for i in range(1,50):
    url = 'https://slickdeals.net/?page=' + str(i)
    url_list.append(url)
#     print(url_list)

In [17]:
sale_items = []

# Trying to loop through pages 1-50 of Slickdeals.net
for url in url_list:
    
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    
    results = soup.find_all('div')
    
    #Create a dataframe to hold
    
    for result in results:
    # Error handling
        try:
            # Identify and return title of listing
            title = result.find('div', class_="priceLine")
            # Identify and return original price of listing
            old_price = result.find('span', class_="oldListPrice")
#             old_price = old_price.text
            # Identify and return new price of listing
            new_price = result.find('div', class_="itemPrice")
#             new_price = new_price.text.strip()
            # Print results only if title,a price, and link are available
            if (title and old_price and new_price):
                print('-------------')
                print("Title")
                print(title["title"])
                print("Old Price")
                print(old_price.text)
                print("New Price")
                print(new_price.text.strip())
                sale_items.append({"Item" : title["title"], "Old Price" : old_price.text, "Sale Price": new_price.text.strip()})
        except AttributeError as e:
            print(e)
            
#     browser.click_link_by_partial_text('Next')

-------------
Title
12oz Birch Benders Pancake & Waffle Mix (Paleo) $3.75
Old Price
$48
New Price
$3.75
-------------
Title
12oz Birch Benders Pancake & Waffle Mix (Paleo) $3.75
Old Price
$48
New Price
$3.75
-------------
Title
12oz Birch Benders Pancake & Waffle Mix (Paleo) $3.75
Old Price
$48
New Price
$3.75
-------------
Title
12oz Birch Benders Pancake & Waffle Mix (Paleo) $3.75
Old Price
$48
New Price
$3.75
-------------
Title
Bosch 1"x 10" SDS-plus Bulldog Rotary Hammer Bit $6.30 &amp; More
Old Price
$48
New Price
$6.30
                                                                                  $48
-------------
Title
Bosch 1"x 10" SDS-plus Bulldog Rotary Hammer Bit $6.30 &amp; More
Old Price
$48
New Price
$6.30
                                                                                  $48
-------------
Title
Bosch 1"x 10" SDS-plus Bulldog Rotary Hammer Bit $6.30 &amp; More
Old Price
$48
New Price
$6.30
                                                                

-------------
Title
Star Wars Men's Graphic Tees (Various Styles) From $2.40 + Free Store Pickup on $25+
Old Price
$20
New Price
From $2.40
-------------
Title
Star Wars Men's Graphic Tees (Various Styles) From $2.40 + Free Store Pickup on $25+
Old Price
$20
New Price
From $2.40
-------------
Title
Star Wars Men's Graphic Tees (Various Styles) From $2.40 + Free Store Pickup on $25+
Old Price
$20
New Price
From $2.40
-------------
Title
Star Wars Men's Graphic Tees (Various Styles) From $2.40 + Free Store Pickup on $25+
Old Price
$20
New Price
From $2.40
-------------
Title
Star Wars Men's Graphic Tees (Various Styles) From $2.40 + Free Store Pickup on $25+
Old Price
$20
New Price
From $2.40
-------------
Title
Star Wars Men's Graphic Tees (Various Styles) From $2.40 + Free Store Pickup on $25+
Old Price
$20
New Price
From $2.40
-------------
Title
Star Wars Men's Graphic Tees (Various Styles) From $2.40 + Free Store Pickup on $25+
Old Price
$20
New Price
From $2.40
-------------
Title


-------------
Title
55" Samsung QN55Q60RA Q60 4K Smart QLED TV + Mounting Kit Bundle $679 + Free Shipping
Old Price
$893
New Price
$679
                                                                                                                            $893
-------------
Title
55" Samsung QN55Q60RA Q60 4K Smart QLED TV + Mounting Kit Bundle $679 + Free Shipping
Old Price
$893
New Price
$679
                                                                                                                            $893
-------------
Title
55" Samsung QN55Q60RA Q60 4K Smart QLED TV + Mounting Kit Bundle $679 + Free Shipping
Old Price
$893
New Price
$679
                                                                                                                            $893
-------------
Title
55" Samsung QN55Q60RA Q60 4K Smart QLED TV + Mounting Kit Bundle $679 + Free Shipping
Old Price
$893
New Price
$679
                                                                     

-------------
Title
1-Year Sony PlayStation Plus Membership (Digital Delivery) $36.80
Old Price
$60
New Price
$36.80
                                                                                                                            $60
-------------
Title
1-Year Sony PlayStation Plus Membership (Digital Delivery) $36.80
Old Price
$60
New Price
$36.80
                                                                                                                            $60
-------------
Title
1-Year Sony PlayStation Plus Membership (Digital Delivery) $36.80
Old Price
$60
New Price
$36.80
                                                                                                                            $60
-------------
Title
1-Year Sony PlayStation Plus Membership (Digital Delivery) $36.80
Old Price
$60
New Price
$36.80
                                                                                                                            $60
-------------
Title


Dr. Scholl’s Air-Pillo Insoles for Men or Women (One-Pair) $1.85
Old Price
$3
New Price
$1.85
                                                                                                                            $3
-------------
Title
Dr. Scholl’s Air-Pillo Insoles for Men or Women (One-Pair) $1.85
Old Price
$3
New Price
$1.85
                                                                                                                            $3
-------------
Title
Asus ROG Strix Desktop: Ryzen 7 3700X, 16GB DDR4, 512GB SSD, RTX 2060 Super $1100 + Free Shipping
Old Price
$1300
New Price
$1100
                                                                                                                            $1300
-------------
Title
Asus ROG Strix Desktop: Ryzen 7 3700X, 16GB DDR4, 512GB SSD, RTX 2060 Super $1100 + Free Shipping
Old Price
$1300
New Price
$1100
                                                                                                            

...

-------------
Title
12oz Birch Benders Pancake & Waffle Mix (Paleo) $3.75
Old Price
$48
New Price
$3.75
-------------
Title
12oz Birch Benders Pancake & Waffle Mix (Paleo) $3.75
Old Price
$48
New Price
$3.75
-------------
Title
12oz Birch Benders Pancake & Waffle Mix (Paleo) $3.75
Old Price
$48
New Price
$3.75
-------------
Title
12oz Birch Benders Pancake & Waffle Mix (Paleo) $3.75
Old Price
$48
New Price
$3.75
-------------
Title
Bosch 1"x 10" SDS-plus Bulldog Rotary Hammer Bit $6.30 &amp; More
Old Price
$48
New Price
$6.30
                                                                                  $48
-------------
Title
Bosch 1"x 10" SDS-plus Bulldog Rotary Hammer Bit $6.30 &amp; More
Old Price
$48
New Price
$6.30
                                                                                  $48
-------------
Title
Bosch 1"x 10" SDS-plus Bulldog Rotary Hammer Bit $6.30 &amp; More
Old Price
$48
New Price
$6.30
                                                                

New Price
$71.30
                                                                                  $130
-------------
Title
Lenovo ThinkCentre M75q Tiny Desktop: Ryzen 5 Pro 3400GE, 32GB DDR4, 512GB SSD $451.55 + Free Shipping
Old Price
$821
New Price
$451.55
                                                                                                                            $821
-------------
Title
Lenovo ThinkCentre M75q Tiny Desktop: Ryzen 5 Pro 3400GE, 32GB DDR4, 512GB SSD $451.55 + Free Shipping
Old Price
$821
New Price
$451.55
                                                                                                                            $821
-------------
Title
Lenovo ThinkCentre M75q Tiny Desktop: Ryzen 5 Pro 3400GE, 32GB DDR4, 512GB SSD $451.55 + Free Shipping
Old Price
$821
New Price
$451.55
                                                                                                                            $821
-------------
Title
Lenovo ThinkCentre M7

-------------
Title
12oz Birch Benders Pancake & Waffle Mix (Paleo) $3.75
Old Price
$48
New Price
$3.75
-------------
Title
12oz Birch Benders Pancake & Waffle Mix (Paleo) $3.75
Old Price
$48
New Price
$3.75
-------------
Title
12oz Birch Benders Pancake & Waffle Mix (Paleo) $3.75
Old Price
$48
New Price
$3.75
-------------
Title
12oz Birch Benders Pancake & Waffle Mix (Paleo) $3.75
Old Price
$48
New Price
$3.75
-------------
Title
Bosch 1"x 10" SDS-plus Bulldog Rotary Hammer Bit $6.30 &amp; More
Old Price
$48
New Price
$6.30
                                                                                  $48
-------------
Title
Bosch 1"x 10" SDS-plus Bulldog Rotary Hammer Bit $6.30 &amp; More
Old Price
$48
New Price
$6.30
                                                                                  $48
-------------
Title
Bosch 1"x 10" SDS-plus Bulldog Rotary Hammer Bit $6.30 &amp; More
Old Price
$48
New Price
$6.30
                                                                

Title
24-Pack 11.15oz S.Pellegrino Essenza Flavored Mineral Water (Lemon & Lemon Zest) $8.20 w/ S&amp;S + Free S&amp;H
Old Price
$9
New Price
$8.20
                                                                                      (5% OFF)
$9
-------------
Title
64-Ounce Rug Doctor Pure Power Oxy Carpet Cleaner $9 + Free Store Pickup
Old Price
$19
New Price
$9
                                                                                      (53% OFF)
$19
-------------
Title
64-Ounce Rug Doctor Pure Power Oxy Carpet Cleaner $9 + Free Store Pickup
Old Price
$19
New Price
$9
                                                                                      (53% OFF)
$19
-------------
Title
64-Ounce Rug Doctor Pure Power Oxy Carpet Cleaner $9 + Free Store Pickup
Old Price
$19
New Price
$9
                                                                                      (53% OFF)
$19
-------------
Title
64-Ounce Rug Doctor Pure Power Oxy Carpet Cleaner $9 + Free Store Pickup
O

-------------
Title
Skip Hop Toddler Backpack: Dinosaur or Unicorn $10 each
Old Price
$22
New Price
$10 each
-------------
Title
Skip Hop Toddler Backpack: Dinosaur or Unicorn $10 each
Old Price
$22
New Price
$10 each
-------------
Title
Skip Hop Toddler Backpack: Dinosaur or Unicorn $10 each
Old Price
$22
New Price
$10 each
-------------
Title
Just Dance 2020 (Nintendo Switch) $18 + Free Store Pickup
Old Price
$40
New Price
$18
                                                                                      (55% OFF)
$40
-------------
Title
Just Dance 2020 (Nintendo Switch) $18 + Free Store Pickup
Old Price
$40
New Price
$18
                                                                                      (55% OFF)
$40
-------------
Title
Just Dance 2020 (Nintendo Switch) $18 + Free Store Pickup
Old Price
$40
New Price
$18
                                                                                      (55% OFF)
$40
-------------
Title
Just Dance 2020 (Nintendo Switch) $

In [18]:
sale_items_df = pd.DataFrame(sale_items)
sale_items_df

,Item,Old Price,Sale Price
0,12oz Birch Benders Pancake & Waffle Mix (Paleo...,$48,$3.75
1,12oz Birch Benders Pancake & Waffle Mix (Paleo...,$48,$3.75
2,12oz Birch Benders Pancake & Waffle Mix (Paleo...,$48,$3.75
3,12oz Birch Benders Pancake & Waffle Mix (Paleo...,$48,$3.75
4,"Bosch 1""x 10"" SDS-plus Bulldog Rotary Hammer B...",$48,$6.30\n ...
...,...,...,...
6111,Star Wars Men's Graphic Tees (Various Styles) ...,$20,From $2.40\n ...
6112,Dyson Cyclone V10 Absolute Vacuum Cleaner with...,$600,$400\n ...
6113,Dyson Cyclone V10 Absolute Vacuum Cleaner with...,$600,$400\n ...
6114,Dyson Cyclone V10 Absolute Vacuum Cleaner with...,$600,$400\n ...


In [41]:
sale_items_df.to_csv("slickdeals.csv", index=True)

In [42]:
master_list_df = sale_items_df
# master_list_df.columns
# pick_up_method.columns

Index(['Item'], dtype='object')

In [43]:
# Split item column after "free"

# leaves "free shipping" and "store pick up"
pick_up_method = pd.DataFrame(sale_items_df.Item.str.split('+').str[1])
pick_up_method.to_csv('slickdeal_items_w_pickup.csv', index=True)



In [44]:
#drops the shipping/store pickup
split_items = pd.DataFrame(sale_items_df.Item.str.split('+').str[0])
split_items.to_csv("slickdeal_items.csv", index=True)

In [47]:
merged_items_df = split_items.merge(pick_up_method, left_index=True, right_index=True, how='inner')

In [49]:
merged_items_df = merged_items_df.rename(columns = {"Item_x" : "Item",
                                  "Item_y" : "Pick Up Notes"})
merged_items_df.to_csv('merged_item_columns.csv', index=True)

In [ ]:
split_items = pd.DataFrame(sale_items_df.Item.str.split('+',1))

In [ ]:
# Clean up data and upload to a database